## Noisy TypeWriter

Consider the noisy typewriter channel, mapping

$$ A_Z = \{ A, B, C, . . . , Y, Z, − \} $$

with $|A_Z| = 27$, into $A_Y = A_Z$, where each letter is mapped with equal probabilities into the preceding, the following or the same letter. Design an efficient code by which to reliably send symbols from $A_X = A_Z$ through the channel (i.e., you should be able to send and retrieve a text using the 27 synbols with no error).  

Write a program implementing the channel and the code, and test it.

In [9]:
import string
import numpy as np
from functools import reduce

In [10]:
possible_characters = [character for character in string.ascii_lowercase] + [' ']
len(possible_characters)

27

In [43]:
len_text = 22

list_text = np.random.choice(possible_characters, size = len_text, replace = True)

text = reduce(lambda x,y: x+y, list_text, '')

text

'rnqglydxrtec ndsplafyf'

### Approach

We will solve the task in the following way:

* We will associate to each letter a number from 9 on in base 9 (we start from 9 in order to have always 2 units).
* We will associate to a group of 3 neighbouring letters a number between 0 and 8.
* When we send the letter we first encode it as the number associated to the letter using for each unit the central letter in the correspondong set of 3 neighbouring characters.
* Decode the message

In [12]:
encoding_to_letter = {str(i):letter for i in range(9) for letter in [possible_characters[i * 3 + 1]]}
encoding_to_letter   #the unique codewords


{'0': 'b',
 '1': 'e',
 '2': 'h',
 '3': 'k',
 '4': 'n',
 '5': 'q',
 '6': 't',
 '7': 'w',
 '8': 'z'}

In [13]:
letter_to_decoding = {possible_characters[i]: str((i) // 3)  for i in range(len(possible_characters))}
letter_to_decoding

{'a': '0',
 'b': '0',
 'c': '0',
 'd': '1',
 'e': '1',
 'f': '1',
 'g': '2',
 'h': '2',
 'i': '2',
 'j': '3',
 'k': '3',
 'l': '3',
 'm': '4',
 'n': '4',
 'o': '4',
 'p': '5',
 'q': '5',
 'r': '5',
 's': '6',
 't': '6',
 'u': '6',
 'v': '7',
 'w': '7',
 'x': '7',
 'y': '8',
 'z': '8',
 ' ': '8'}

In [14]:
letter_to_codeword = {possible_characters[i]: np.base_repr(i + 9, 9) for i in range(len(possible_characters))}
letter_to_codeword


{'a': '10',
 'b': '11',
 'c': '12',
 'd': '13',
 'e': '14',
 'f': '15',
 'g': '16',
 'h': '17',
 'i': '18',
 'j': '20',
 'k': '21',
 'l': '22',
 'm': '23',
 'n': '24',
 'o': '25',
 'p': '26',
 'q': '27',
 'r': '28',
 's': '30',
 't': '31',
 'u': '32',
 'v': '33',
 'w': '34',
 'x': '35',
 'y': '36',
 'z': '37',
 ' ': '38'}

In [15]:
codeword_to_letter = {np.base_repr(i + 9, 9): possible_characters[i] for i in range(len(possible_characters))}


In [36]:
def encoder(letter):
    assert letter in possible_characters, "Letter not in possible characters"

    number_to_send = letter_to_codeword[letter]

    first_unit = number_to_send[0]
    second_unit = number_to_send[1]

    return encoding_to_letter[first_unit] + encoding_to_letter[second_unit]


In [37]:
def channel(input):
    output = ''

    for letter in input:
        assert letter in possible_characters, "Letter not in possible characters"

        letter_index = possible_characters.index(letter)
        possible_letters = [possible_characters[i % len(possible_characters)] for i in [letter_index - 1, letter_index, letter_index + 1]]
        
        output += str(np.random.choice(possible_letters, size = 1)[0])

    return output


In [38]:
def decoder(input):

    final_decoding = ''
    
    for letter in input:
        assert letter in possible_characters, "Letter not in possible characters"

        final_decoding += letter_to_decoding[letter]

    return codeword_to_letter[final_decoding] 


In [42]:

# Verifying the decoding of the encoded chars
for character in possible_characters:
    print(f'''{character} -> {decoder(channel(encoder(character)))}''')
    

a -> a
b -> b
c -> c
d -> d
e -> e
f -> f
g -> g
h -> h
i -> i
j -> j
k -> k
l -> l
m -> m
n -> n
o -> o
p -> p
q -> q
r -> r
s -> s
t -> t
u -> u
v -> v
w -> w
x -> x
y -> y
z -> z
  ->  


In [52]:
# Visualizing the process of encoding and decoding for our text
[('x =  ' + letter + '    encoding in ->  ' + encoder(letter) + '   noisy channel ->   ' +  channel(encoder(letter)) 
  +  '   decoding in ->   ' +  decoder(channel(encoder(letter)))) for letter in text]

['x =  r    encoding in ->  hz   noisy channel ->   h    decoding in ->   r',
 'x =  n    encoding in ->  hn   noisy channel ->   ho   decoding in ->   n',
 'x =  q    encoding in ->  hw   noisy channel ->   hw   decoding in ->   q',
 'x =  g    encoding in ->  et   noisy channel ->   du   decoding in ->   g',
 'x =  l    encoding in ->  hh   noisy channel ->   ig   decoding in ->   l',
 'x =  y    encoding in ->  kt   noisy channel ->   lt   decoding in ->   y',
 'x =  d    encoding in ->  ek   noisy channel ->   el   decoding in ->   d',
 'x =  x    encoding in ->  kq   noisy channel ->   lq   decoding in ->   x',
 'x =  r    encoding in ->  hz   noisy channel ->   hy   decoding in ->   r',
 'x =  t    encoding in ->  ke   noisy channel ->   ke   decoding in ->   t',
 'x =  e    encoding in ->  en   noisy channel ->   dn   decoding in ->   e',
 'x =  c    encoding in ->  eh   noisy channel ->   fh   decoding in ->   c',
 'x =       encoding in ->  kz   noisy channel ->   j    decodin